<a href="https://colab.research.google.com/github/shimizu-yuta-intern/yatin_data/blob/main/%E5%AE%B6%E8%B3%83%E3%83%87%E3%83%BC%E3%82%BF%E5%8F%8E%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import requests
from bs4 import BeautifulSoup
import retry
import urllib
import time
import numpy as np

# 複数ページの情報をまとめて取得
data_samples = []

# スクレイピングするページ数
max_page = 2000

# SUUMOを東京都23区のみ指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=50&page={}'

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1, max_page + 1):
    before = time.time()
    try:
        # ページ情報
        soup = load_page(url.format(page))
        # 物件情報リストを指定
        mother = soup.find_all(class_='cassetteitem')

        # 物件ごとの処理
        for child in mother:
            # 建物情報
            data_home = []
            try:
                # カテゴリ
                data_home.append(child.find(class_='ui-pct ui-pct--util1').text.strip())
                # 建物名
                data_home.append(child.find(class_='cassetteitem_content-title').text.strip())
                # 住所
                data_home.append(child.find(class_='cassetteitem_detail-col1').text.strip())
                # 最寄り駅のアクセス
                children = child.find(class_='cassetteitem_detail-col2')
                for grandchild in children.find_all(class_='cassetteitem_detail-text'):
                    data_home.append(grandchild.text.strip())
                # 築年数と階数
                children = child.find(class_='cassetteitem_detail-col3')
                for grandchild in children.find_all('div'):
                    data_home.append(grandchild.text.strip())
            except AttributeError:
                continue  # 必要な情報が取得できなかった場合はスキップする

            # 部屋情報
            rooms = child.find(class_='cassetteitem_other')
            for room in rooms.find_all(class_='js-cassette_link'):
                data_room = []
                try:
                    # 部屋情報が入っている表を探索
                    for id_, grandchild in enumerate(room.find_all('td')):
                        # 階
                        if id_ == 2:
                            data_room.append(grandchild.text.strip())
                        # 家賃と管理費
                        elif id_ == 3:
                            data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text.strip())
                            data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text.strip())
                        # 敷金と礼金
                        elif id_ == 4:
                            data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text.strip())
                            data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text.strip())
                        # 間取りと面積
                        elif id_ == 5:
                            data_room.append(grandchild.find(class_='cassetteitem_madori').text.strip())
                            data_room.append(grandchild.find(class_='cassetteitem_menseki').text.strip())
                        # URL
                        elif id_ == 8:
                            get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href', None)
                            if get_url:
                                abs_url = urllib.parse.urljoin(url, get_url)
                                data_room.append(abs_url)
                except AttributeError:
                    continue  # 必要な情報が取得できなかった場合はスキップする

                # 物件情報と部屋情報をくっつける
                data_sample = data_home + data_room
                data_samples.append(data_sample)
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        continue

    # 1アクセスごとに1秒休む
    time.sleep(1)

    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data_samples)}')
    # 作業進捗
    complete_ratio = round(page / max_page * 100, 3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time / 3600)
    minute = int((running_required_time % 3600) / 60)
    second = int(running_required_time % 60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：', running_all)

ModuleNotFoundError: No module named 'retry'